# PyTerrier - Neural Re-Ranking 

Dans ce TP vous allez :

 - reclasser des documents en utilisant des modèles neuronaux comme KNRM, Vanilla BERT, EPIC et monoT5.


# Installation


In [1]:
!pip install -q --upgrade python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 

### Installation des plugins Pyterrier  

Nous installons les plugins PyTerrier [OpenNIR](https://opennir.net/) et [monoT5](https://github.com/terrierteam/pyterrier_t5). Vous pouvez ignorer sans risque les erreurs de version des paquets.

In [2]:
!pip install -q --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.1 MB/s eta

## Preliminary steps

In [3]:
import os
import tensorflow as tf

**[PyTerrier](https://github.com/terrier-org/pyterrier) initialization** 

In [4]:
import pyterrier as pt
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


from pyterrier.measures import * # allow for natural measure names
import onir_pt

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Indexation

In [5]:
!rm -rf ./terrier_cord19/*

In [6]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 s<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M s<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 65.5k/269M s<10:34, 425kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.1%| 254k/269M s<05:29, 817kB/s] 
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.4%| 950k/269M s<02:13, 2.02MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

<ipython-input-6-59cedea60614>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


20:43:31.911 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
20:44:42.803 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

## Re-Rankers 

Commençons à explorer quelques méthodes neuronales de re-classement ! Nous pouvons les construire à partir de zéro en utilisant `onir_pt.reranker`.

Le modèle de re-ranking d'OpenNIR est composé de :
 - `ranker` (par exemple, `drmm`, `knrm`, ou `pacrr`). Ceci définit l'architecture neuronale de classement.
 - `vocab` (par exemple, `wordvec_hash`, ou `bert`). Ceci définit comment le texte est encodé par le modèle. Cette approche rend facile l'échange de différentes représentations de texte.

L'exécution de cette ligne prendra quelques minutes car elle télécharge et prépare les vecteurs de mots.

In [7]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2023-03-31 20:44:55,905][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2023-03-31 20:45:01,714][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2023-03-31 20:45:01,721][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [5.73s] [682M] [117MB/s]
[2023-03-31 20:45:01,727][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [5.82s]
[2023-03-31 20:45:01,727][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2023-03-31 20:45:15,388][WordvecHashVocab][DEBUG] [finished] extracting vecs [13.66s]
[2023-03-31 20:45:15,389][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2023-03-31 20:47:42,095][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:27]
[2023-03-31 20:47:42,345][WordvecHashVocab][DEBUG] [starting] writing

Voyons comment ces modèles fonctionnent pour le classement !

In [8]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(cord19, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])


In [9]:
topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [18.7kB] [9.96MB/s]
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: s] [1.14MB] [6.77MB/s]


In [10]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

In [11]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2023-03-31 20:48:02,292][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:48:07,375][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2023-03-31 20:48:10,887][onir_pt][DEBUG] [finished] batches: [3.51s] [188it] [53.55it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM,0.042539,0.112699,0.423870,0.360000


Cela ne fonctionne pas très bien car le modèle n'est pas entraîné ; il utilise des poids aléatoires pour combiner les scores de la matrice de similarité.

## Entraînement du re-ranker

Vous pouvez entraîner des modèles de re-classement dans PyTerrier en utilisant la méthode `fit`. 

In [12]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

[2023-03-31 20:48:14,455][onir_pt][DEBUG] [starting] validation
[2023-03-31 20:48:14,457][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:48:14,458][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2023-03-31 20:48:14,618][onir_pt][DEBUG] [finished] batches: s] [63it] [394.83it/s]
[2023-03-31 20:48:14,734][onir_pt][DEBUG] [finished] validation s]
[2023-03-31 20:48:14,735][onir_pt][INFO] pre-validation: 0.0053
[2023-03-31 20:48:14,752][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:48:14,752][onir_pt][DEBUG] [starting] training
[2023-03-31 20:48:14,753][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2023-03-31 20:48:15,018][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:15,530][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:16,639][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:17,850][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:18,335][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:19,442][onir_pt][DEBUG] not enough negs
[2023-03-31 20:48:20,708][onir_pt][DEBUG] [finished] train pairs: [5.96s] [1024it] [171.95it/s]
[2023-03-31 20:48:20,712][onir_pt][DEBUG] [finished] training [5.96s]
[2023-03-31 20:48:20,713][onir_pt][INFO] training   it=0 loss=0.2451
[2023-03-31 20:48:20,713][onir_pt][DEBUG] [starting] validation
[2023-03-31 20:48:20,714][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:48:20,715][onir_pt][DEBUG] [starting] batches


...

[2023-03-31 20:52:01,757][onir_pt][DEBUG] [finished] batches: s] [63it] [297.98it/s]
[2023-03-31 20:52:01,960][onir_pt][DEBUG] [finished] validation s]
[2023-03-31 20:52:01,962][onir_pt][INFO] validation it=36 map=0.0060 ndcg=0.0153 P_10=0.0720
[2023-03-31 20:52:01,963][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:52:01,964][onir_pt][DEBUG] [starting] training
[2023-03-31 20:52:01,964][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2023-03-31 20:52:02,233][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:03,410][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:04,619][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:05,355][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:05,498][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:06,174][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:06,226][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:07,083][onir_pt][DEBUG] not enough negs
[2023-03-31 20:52:07,362][onir_pt][DEBUG] [finished] train pairs: [5.40s] [1024it] [189.73it/s]
[2023-03-31 20:52:07,366][onir_pt][DEBUG] [finished] training [5.40s]
[2023-03-31 20:52:07,367][onir_pt][INFO] training   it=37 loss=0.2167
[2023-03-31 20:52:07,368][onir_pt][DEBUG] [starting] validation
[2023-03-31 20:52:07,368][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:52:07,369][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2023-03-31 20:52:07,616][onir_pt][DEBUG] [finished] batches: s] [63it] [255.91it/s]
[2023-03-31 20:52:07,893][onir_pt][DEBUG] [finished] validation s]
[2023-03-31 20:52:07,895][onir_pt][INFO] validation it=37 map=0.0059 ndcg=0.0152 P_10=0.0700
[2023-03-31 20:52:07,895][onir_pt][INFO] early stopping; model reverting back to it=17


In [13]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2023-03-31 20:52:10,966][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:52:10,968][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2023-03-31 20:52:11,695][onir_pt][DEBUG] [finished] batches: s] [188it] [259.01it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM (trained),0.051552,0.121711,0.506577,0.420000


In [26]:
#je n'arrive pas a diminuer la taille des donnees, j'ai essaye plusieurs methodes mais aucune n'a fonctionne. Je ne peux pas tester le code sur google colab 

Le résultat est plus élévé, mais cela reste moins bon que le TFIDF. Proposez une hypothèse sur le problème.

## Vanilla BERT

Les modèles linguistiques contextualisés, tels que [BERT] (https://arxiv.org/abs/1810.04805), sont des modèles neuronaux beaucoup plus puissants qui se sont avérés efficaces pour le classement.

Nous allons essayer d'utiliser une version "vanille" (ou "mono") du modèle BERT. Le modèle BERT est pré-entraîné pour la modélisation du langage et la prédiction de la phrase suivante.

In [14]:
#del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

100%|██████████| 231508/231508 s<0ms, 22230704.24B/s]
100%|██████████| 433/433 s<0ms, 280917.81B/s]
100%|██████████| 440473133/440473133 [9.97s<0ms, 44191589.33B/s] 


Voyons comment ce modèle se comporte sur la collection TREC COVID.

In [15]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2023-03-31 20:52:35,540][onir_pt][DEBUG] using GPU (deterministic)
[2023-03-31 20:52:35,696][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2023-03-31 20:52:56,451][onir_pt][DEBUG] [finished] batches: [20.75s] [188it] [ 9.06it/s]


,name,nDCG,nDCG@10,AP(rel=2),P(rel=2)@10,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,AP(rel=2) +,AP(rel=2) -,AP(rel=2) p-value,P(rel=2)@10 +,P(rel=2)@10 -,P(rel=2)@10 p-value
0,TFIDF,0.123589,0.595818,0.054568,0.546667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TFIDF >> VBERT,0.118277,0.479929,0.049198,0.420000,3.0,12.0,0.11542,4.0,11.0,0.02287,3.0,12.0,0.012239,3.0,12.0,0.024362


Comme nous le constatons, bien que le modèle soit pré-entraîné, il n'obtient pas de très bons résultats. Cela est dû au fait qu'il n'est pas réglé pour la tâche de classement par pertinence.

Cependant, nous pouvons entraîner le modèle pour le classement (comme indiqué ci-dessus pour KNRM).

## monoT5

Le modèle [monoT5](https://arxiv.org/abs/2003.06713) évalue les documents à l'aide d'un modèle de langage causal. Voyons comment cette approche fonctionne sur TREC COVID.

La classe `MonoT5ReRanker` de `pyterrier_t5` charge automatiquement une version du classeur monoT5 qui est entraînée sur le jeu de données MS MARCO passage.

In [16]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
pipeline = (tfidf >> get_text >> monoT5)
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

monoT5:   0%|          | 0/188 s<?, ?batches/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10,mrt
0,TFIDF,0.054568,0.123589,0.595818,0.546667,52.372967
1,TFIDF >> T5,0.058684,0.128119,0.685513,0.600000,1701.459983


Comme on pouvait s'y attendre, les résultats sont bien meilleurs en termes de NDCG@10 (0.5958 vs 0.6855).

# Tâche pratique

Comme pour le TP precedent, utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Vous pouvez le télécharger le dataset en utilisant les lines de code ci-dessous.

In [18]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train')
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/17918ed23cd04fb15047f73e6c3bd9d9
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip
[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: [1.06s] [17.9MB] [16.9MB/s]
[INFO] [finished] opening zip file [1.56s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


In [28]:
!pip install -q --upgrade fastrank lightgbm==3.1.1
!pip install -q python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.8 MB/s eta 0:00:00


In [36]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import fastrank
RANK_CUTOFF = 10
SEED=42
ltr_feats1 = (tfidf % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> tfidf)
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='TF_IDF') ) 
    ** # date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** # has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    ** # abstract coordinate match
    pt.BatchRetrieve(index_ref, wmodel="CoordinateMatch")
)

In [34]:
def append_baseline(system, baseline, max_results = 1000):
    return (system ^ baseline) % max_results

In [ ]:
train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, train_qrels)

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf, form='regression')


%time rf_pipe.fit(train_topics, train_qrels)
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, train_qrels, valid_topics, valid_qrels)
pt.Experiment(
    [tfidf % RANK_CUTOFF, ca_pipe, rf_pipe, lmart_x_pipe],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

lmart_rf= append_baseline(lmart_x_pipe,rf_pipe)

lmart_ca = append_baseline(lmart_x_pipe,ca_pipe)
rf_ca = append_baseline(rf_pipe,ca_pipe)
pt.Experiment(
    [tfidf % RANK_CUTOFF, lmart_ca, lmart_rf, rf_ca],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

21:48:11.117 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f52a81bbfd0>, expected 54959 received 55321, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


Identifiez-vous des avantages par rapport à l'utilisation de Learning to Rank ?